In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Load the train data (referred as data)
data = pd.read_csv("train.csv")
# Load the test data
test_data = pd.read_csv("test.csv")

# Initial look at the data structure
print("Data Shape:", data.shape)
print(data.info())
print(data.isnull().sum())  # Check for missing values

print("Test Data Shape:", test_data.shape)
print(test_data.info())
print(test_data.isnull().sum())  # Check for missing values

In [ ]:
# Impute missing values
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')

# Separate numeric and categorical columns
numeric_features = data.select_dtypes(include=[np.number]).columns
categorical_features = data.select_dtypes(include=[object]).columns
# Separate numeric and categorical columns
numeric_features_test = test_data.select_dtypes(include=[np.number]).columns
categorical_features_test = test_data.select_dtypes(include=[object]).columns

# Apply imputers
data[numeric_features] = imputer_num.fit_transform(data[numeric_features])
data[categorical_features] = imputer_cat.fit_transform(data[categorical_features])

# Apply imputers
test_data[numeric_features_test] = imputer_num.fit_transform(test_data[numeric_features_test])
test_data[categorical_features_test] = imputer_cat.fit_transform(test_data[categorical_features_test])

print("Missing values after imputation:", data.isnull().sum().sum())
print("Missing test values after imputation:", test_data.isnull().sum().sum())

In [ ]:
le = LabelEncoder()
# Label encode categorical features
for col in categorical_features:
    data[col] = le.fit_transform(data[col])

print("Data with encoded categorical features:")
print(data.head())

# Label encode categorical features
for col in categorical_features_test:
    test_data[col] = le.fit_transform(test_data[col])

print("Test data with encoded categorical features:")
print(test_data.head())

In [ ]:
# Define target and features
X = data.drop(columns=['SalePrice'])
y = np.log1p(data['SalePrice'])  # Applying log transformation to target
X_final = test_data

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=900)

# Align the train and test data
X_train, X_test = X_train.align(X_test, join='inner', axis=1)
# Scale the features
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(X_train)
test_data_scaled = scaler.transform(X_test)
test_data_final = scaler.transform(X_final)

print("Data processing complete.")

print("Training data shape:", train_data_scaled.shape)
print("Testing data shape:", test_data_scaled.shape)
print("Final testing data shape:", test_data_final.shape)

In [ ]:
# [80][0.20000000000000004][34][3]
import random as r
min = 1
# Number of predictors
# for i in range(40,81):
    # learning rate
j = 0.11
while(j < 0.22):
    # random state
    for k in range(50,70,r.randint(5,10)):
        # Depth
        # for l in range(1,4):
            # Initialize and train XGBoost Model
            xgb_model = xgb.XGBRegressor(n_estimators=80, max_depth=3, learning_rate=j, random_state=k)
            xgb_model.fit(train_data_scaled, y_train)

            # Evaluate model
            y_pred = xgb_model.predict(test_data_scaled)
            predictions = xgb_model.predict(test_data_final)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            j += 0.007
            if(rmse < min):
                min = rmse
                print(f"Root Mean Squared Error (RMSE) [{i}][{j}][{k}][{l}]:", rmse)
print("Finished !!!")

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators=80, max_depth=3, learning_rate=0.2150000000000008, random_state=50)
xgb_model.fit(train_data_scaled, y_train)

# Evaluate model
y_pred = xgb_model.predict(test_data_scaled)
predictions = xgb_model.predict(test_data_final)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Root Mean Squared Error (RMSE):", rmse)

# generating the submission.csv file using sample_submission.csv file
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['SalePrice'] = predictions
submission.to_csv('submission_x.csv', index=False)
print("Submission file created: submission_x.csv")

In [ ]:
# Read the submission.csv file
df = pd.read_csv("submission_x.csv")

# Multiply each value in the "SalePrice" column by 10000
df["SalePrice"] = df["SalePrice"] * 10000

# Save the modified data to a new file
df.to_csv("submission.csv", index=False)

print("Submission files has been created with updated SalePrice values.")
